## VarQRTE Error

In [15]:
from jax import grad, jit
import jax.numpy as jnp

import os
import warnings

import numpy as np
import scipy as sp
from scipy.linalg import expm

from qiskit.circuit.library import EfficientSU2, RealAmplitudes

from qiskit.opflow import StateFn, MatrixOp, CircuitOp, X, I, PauliOp, Z, Y, SummedOp
from qiskit.opflow.gradients import NaturalGradient
from qiskit.opflow.evolutions.varqtes.varqrte import VarQRTE

np.random.seed = 2

In [16]:
# Set Hamiltonian
# Hamiltonian = SummedOp([(Z ^ X), 0.3 *(Y ^ Y), (Y ^ I)]).reduce()
Hamiltonian = SummedOp([(Z ^ X), 0.8 *(Y ^ Y)]).reduce() # works
Hamiltonian = SummedOp([(Z ^ X), (X ^ Z), 3 * (Z ^ Z)]).reduce()
# Hamiltonian = SummedOp([(Z ^ X), 3. * (Y ^ Y), (Z ^ X), (I ^ Z), (Z ^ I)]).reduce() #works
# Hamiltonian = (Z ^ X) # works
# Hamiltonian = (Y ^ I)
# Set time and time_steps 
time = 1
time_steps = 20

In [17]:
# Helper Function computing the inner product
def inner_prod(x, y):
    return np.matmul(np.conj(np.transpose(x)), y)

In [18]:
# Set Ansatz and initial parameters
ansatz = EfficientSU2(2, reps=1, entanglement='linear')
parameters = ansatz.ordered_parameters
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
    
    

### Analytic Calculations

In [19]:
def analytic_error(state, H, et_ket=None):
    if et_ket is None:
        et_ket = np.zeros(len(state))
        
    dt_state = -1j*(np.dot(H, state) + et_ket)

    # <dt|dt>
    et = inner_prod(dt_state, dt_state) 
    #<H^2>
    et += inner_prod(state, np.matmul(np.matmul(H, H), state))
    # 2Im<dt|H|state>
    et -= 2 * np.imag(inner_prod(dt_state, np.matmul(H, state)))
    
    print('Gradient error', np.round(np.sqrt(et), 5))
    return np.sqrt(et), dt_state

In [20]:
num_qubits = Hamiltonian.num_qubits
H = Hamiltonian.to_matrix(massive=True)

# Propagation Unitary
evolution_op = lambda t: expm(-1 * H * t)
# Initial State
init_state = StateFn(ansatz).assign_parameters(dict(zip(parameters, init_param_values))).eval().primitive

evolution_op = lambda t: expm(-1j * H * t)
target_state = lambda t: np.dot(evolution_op(t), init_state)
dt_target_state = lambda t: np.dot(-1j * H, target_state(t))


error = 0
prepared_state = target_state(0)
for j in range(0, time_steps):
    et_ket = np.ones(2 ** num_qubits) / np.sqrt(2 ** num_qubits) * j * 1e-4

    et, dt_prepared_state = analytic_error(prepared_state, H, et_ket=et_ket)
    # Euler State propagation
    prepared_state += time/time_steps * dt_prepared_state
    # Compute Error
#     error += time/time_steps * et
    error += time/time_steps * np.sqrt(np.linalg.norm(et_ket))

print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 3))

# sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
print('True error ', np.sqrt(np.linalg.norm(prepared_state - target_state(time))))


Gradient error 0j
Gradient error (0.0001+0j)
Gradient error (0.0002+0j)
Gradient error (0.0003+0j)
Gradient error (0.0004-0j)
Gradient error (0.0005+0j)
Gradient error (0.0006+0j)
Gradient error (0.0007+0j)
Gradient error (0.0008+0j)
Gradient error (0.0009-0j)
Gradient error (0.001+0j)
Gradient error (0.0011+0j)
Gradient error (0.0012+0j)
Gradient error (0.0013-0j)
Gradient error (0.0014-0j)
Gradient error (0.0015-0j)
Gradient error (0.0016-0j)
Gradient error (0.0017+0j)
Gradient error (0.0018+0j)
Gradient error (0.0019+0j)
error bound for time 1.0 0.029
True error  0.5670878871401596


### Numpy Calculations

In [21]:
def dt_params(a, c, regularization=None):
    a = np.real(a)
    c = np.real(c)
    if regularization:
        # If a regularization method is chosen then use a regularized solver to
        # construct the natural gradient.
        nat_grad = NaturalGradient._regularized_sle_solver(
            a, c, regularization=regularization)
    else:
        nat_grad = np.linalg.lstsq(a, c, rcond=None)[0]
        if np.linalg.norm(nat_grad) < 1e-8:
            nat_grad = NaturalGradient._regularized_sle_solver(a,
                                                               c,
                                                               regularization='perturb_diag')
            warnings.warn(r'Norm of the natural gradient smaller than $1e^{-8}$ use '
                          r' `perturb_diag` regularization.')
        if np.linalg.norm(nat_grad) > 1e-4:
            nat_grad = NaturalGradient._regularized_sle_solver(a,
                                                               c,
                                                               regularization='ridge')
            warnings.warn(r'Norm of the natural gradient bigger than $1e^{3}$ use '
                          r' `ridge` regularization.')
#         try:
#             # Try to solve the system of linear equations Ax = C.
#             nat_grad = np.linalg.solve(a, c)
#         except np.linalg.LinAlgError:  # singular matrix
#             print('Singular matrix lstsq solver required')
#             nat_grad, resids, _, _ = np.linalg.lstsq(a, c)
#             print('Residuals', resids)
    return np.real(nat_grad)

In [22]:
def numpy_error(a, c, dt_weights, H, state):

    dtdt = inner_prod(dt_weights, np.matmul(a, dt_weights))
    et = dtdt
    print('dtdt', dtdt)
    
    h_squared = inner_prod(state, np.matmul(np.matmul(H, H), state))
    
    if h_squared < dtdt:
        print('Eq. 8 violated')
    et = np.add(et, h_squared)
    
    print('H^2', h_squared)

    
    dt = 2*inner_prod(dt_weights, c)
    et -= dt
    print('2Im<dt|H|>', dt)
    
    print('Grad error', np.round(np.sqrt(et), 6))
    print('|adtw-c|', np.linalg.norm(np.matmul(a, dt_weights)-c))
    return np.sqrt(et)

In [23]:
print('Warning: Numpy Calculations are only compatible with SU2 depth 1 - else hard-coded changes needed.')

def ry(theta):
    return jnp.array([[jnp.cos(theta/2.), -1*jnp.sin(theta/2)], [jnp.sin(theta/2),
                                                                 jnp.cos(theta/2)]])

def rz(theta):
    return jnp.array([[jnp.exp(-1j * theta / 2.), 0], [0, jnp.exp(1j * theta / 2.)]])


def ryry(alpha, beta):
    return jnp.kron(ry(alpha), ry(beta))

def rzrz(alpha, beta):
    return jnp.kron(rz(alpha), rz(beta))

cx = jnp.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
i = jnp.eye(2)
y = jnp.array([[0, -1j], [1j, 0]])
z = jnp.array([[1, 0], [0, -1]])
iy = -0.5j * jnp.kron(i, y)
yi = -0.5j * jnp.kron(y, i)
iz = -0.5j * jnp.kron(i, z)
zi = -0.5j * jnp.kron(z, i)

init = jnp.array([1, 0, 0, 0])

def state_fn(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec

def state0(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[0]

def state1(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[1]

def state2(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[2]

def state3(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[3]


def A(vec, gradient):
    vec = np.reshape(vec, (len(vec), 1))
    a = np.real((inner_prod(gradient, gradient)))
#     print('a carrying part', np.round(a, 3))
    a = np.subtract(a, np.real(np.matmul(inner_prod(gradient, vec),
                    np.transpose(np.conj(inner_prod(gradient, vec))))))
#     print('a phase fix', np.round(np.real(np.matmul(inner_prod(gradient, vec),
#                     np.transpose(np.conj(inner_prod(gradient, vec))))), 3))
    return a

def C(vec, gradient, h):
    vec = np.reshape(vec, (len(vec), 1))
    c = np.imag(inner_prod(gradient, np.matmul(h, vec)))
    print('carrying part c', np.round(c, 3))
    c = np.add(c, np.real(1j * inner_prod(gradient, vec) * inner_prod(vec, np.matmul(h, vec))))
    print('phase fix c', np.round(np.real(1j * inner_prod(gradient, vec) * inner_prod(vec, np.matmul(h, vec))), 3))
#     print('c', c)
    return c

def grad0(params):
    try:
        return grad(state0)(params)
    except Exception:
        return grad(state0, holomorphic=True)(jnp.complex64(params))
def grad1(params):
    try:
        return grad(state1)(params)
    except Exception:
        return grad(state1, holomorphic=True)(jnp.complex64(params))
def grad2(params):
    try:
        return grad(state2)(params)
    except Exception:
        return grad(state2, holomorphic=True)(jnp.complex64(params))
def grad3(params):
    try:
        return grad(state3)(params)
    except Exception:
        return grad(state3, holomorphic=True)(jnp.complex64(params))


In [24]:
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
params = init_param_values
error = 0

for j in range(time_steps):

    # dim vec x num_params
    gradient = [grad0(params), grad1(params), grad2(params), grad3(params)]
    gradient = np.array([[complex(item) for item in g] for g in gradient]).astype(
        np.complex)

    state = state_fn(params)
    state = np.array([complex(s) for s in state]).astype(np.complex)

    metric = A(state, gradient)
    c_grad = C(state, gradient, H)
    print('grad res', np.round(c_grad*2, 3))
    print('metric', np.round(metric, 3))

    dt_weights = dt_params(metric, c_grad, 'ridge')
    print('dt_weights', dt_weights)

    et = numpy_error(metric, c_grad, dt_weights, H, state)

    error += time/time_steps * et

    params += time/time_steps * np.reshape(dt_weights, np.shape(params))
    print('params', params)
    print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 3))

# sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
print('True error ', np.sqrt(np.linalg.norm(state_fn(params) - target_state(time))))
print(state_fn(params))
print(target_state(time))

carrying part c [[0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [0.5]]
phase fix c [[ 0.]
 [ 0.]
 [-0.]
 [-0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
grad res [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]]
metric [[0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25]]
dt_weights [[-2.99382911e-02]
 [-1.51480624e-02]
 [-4.53246306e-07]
 [-4.53246306e-07]
 [-2.99382911e-02]
 [-1.51480624e-02]
 [ 1.93856517e+00]
 [ 1.93856517e+00]]
dtdt [[1.87969485]]
H^2 (10.999998688697854+0j)
2Im<dt|H|> [[3.87712988]]
Grad error [[3.000427+0.j]]
|adtw-c| 0.026430080603239682
params [-1.49691456e-03 -7.57403119e-04 -2.26623153e-08 -2.26623153e-08
  1.56929941e+00  1.57003892e+00  9.69282586e-02  9.69282586e-02]
error bound fo

carrying part c [[ 0.603]
 [ 0.002]
 [-0.014]
 [ 0.507]
 [ 0.006]
 [ 0.006]
 [ 0.359]
 [ 0.358]]
phase fix c [[ 0.   ]
 [ 0.   ]
 [-0.471]
 [-0.502]
 [ 0.   ]
 [ 0.   ]
 [ 0.01 ]
 [ 0.01 ]]
grad res [[ 1.205]
 [ 0.003]
 [-0.972]
 [ 0.009]
 [ 0.011]
 [ 0.011]
 [ 0.738]
 [ 0.737]]
metric [[ 0.25   0.    -0.    -0.     0.003  0.    -0.     0.   ]
 [ 0.     0.25   0.     0.    -0.     0.235 -0.    -0.   ]
 [-0.     0.     0.03  -0.     0.    -0.     0.    -0.   ]
 [-0.     0.    -0.     0.     0.     0.     0.001  0.003]
 [ 0.003 -0.     0.     0.     0.25  -0.087 -0.     0.003]
 [ 0.     0.235 -0.     0.    -0.087  0.25   0.003  0.   ]
 [-0.    -0.     0.     0.001 -0.     0.003  0.25   0.087]
 [ 0.    -0.    -0.     0.003  0.003  0.     0.087  0.25 ]]
dt_weights [[ 2.14175316]
 [ 0.22002143]
 [-0.66966721]
 [ 0.01419268]
 [ 0.26344593]
 [ 0.13591734]
 [ 1.15748824]
 [ 1.16314184]]
dtdt [[2.11306777]]
H^2 (11.148976399859858+0j)
2Im<dt|H|> [[4.94800627]]
Grad error [[2.883407+0.j]]
|adtw-

carrying part c [[ 1.6  ]
 [-0.213]
 [ 0.039]
 [ 0.424]
 [-0.039]
 [-0.023]
 [ 0.035]
 [-0.066]]
phase fix c [[ 0.   ]
 [ 0.   ]
 [ 0.161]
 [-0.413]
 [-0.03 ]
 [-0.   ]
 [-0.026]
 [-0.046]]
grad res [[ 3.2  ]
 [-0.426]
 [ 0.4  ]
 [ 0.023]
 [-0.138]
 [-0.047]
 [ 0.016]
 [-0.224]]
metric [[ 0.25  -0.     0.     0.    -0.017 -0.     0.019  0.   ]
 [-0.     0.25  -0.     0.     0.001 -0.097  0.001  0.001]
 [ 0.    -0.     0.212  0.     0.007 -0.     0.006 -0.004]
 [ 0.     0.     0.     0.003 -0.018 -0.    -0.016 -0.026]
 [-0.017  0.001  0.007 -0.018  0.249 -0.153 -0.001  0.169]
 [-0.    -0.097 -0.    -0.    -0.153  0.25   0.17  -0.   ]
 [ 0.019  0.001  0.006 -0.016 -0.001  0.17   0.249  0.152]
 [ 0.     0.001 -0.004 -0.026  0.169 -0.     0.152  0.247]]
dt_weights [[ 6.39540809]
 [-0.9931718 ]
 [ 0.86708602]
 [ 0.13048483]
 [ 0.77568976]
 [-0.35655391]
 [ 0.51668233]
 [-1.25987794]]
dtdt [[10.67670828]]
H^2 (11.454554350649676+0j)
2Im<dt|H|> [[21.43831419]]
Grad error [[0.832435+0.j]]
|adt

### Variational Calculations

In [25]:
import os
from qiskit import Aer
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
op = ~StateFn(Hamiltonian)@StateFn(ansatz)
backend = Aer.get_backend('statevector_simulator')
# op = ~StateFn(ansatz)@Hamiltonian@StateFn(ansatz)
op = time * op
approx_time_evolved_state = VarQRTE(parameters=parameters,
                                    grad_method='lin_comb',
                                    init_parameter_values=init_param_values, num_time_steps=time_steps,
                                    regularization='ridge', backend=backend,
                                    error_based_ode=False,
                                    snapshot_dir=os.path.join('dummy')).convert(op)

nat grad result [-2.99380219e-02 -1.51479245e-02  3.15308455e-16  3.79450940e-16
 -2.99380219e-02 -1.51479245e-02  1.93856574e+00  1.93856574e+00]
hsquared 11.0
dtdt 1.8796961640963987
metric [[ 0.25  0.   -0.   -0.    0.   -0.    0.    0.  ]
 [ 0.    0.25 -0.    0.    0.    0.25  0.    0.  ]
 [ 0.   -0.    0.   -0.    0.   -0.   -0.   -0.  ]
 [ 0.   -0.   -0.    0.   -0.   -0.    0.   -0.  ]
 [ 0.    0.   -0.   -0.    0.25  0.    0.    0.  ]
 [ 0.    0.25 -0.    0.    0.    0.25  0.    0.  ]
 [ 0.    0.    0.   -0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.25]]
nat grad result [-0.03  -0.015  0.     0.    -0.03  -0.015  1.939  1.939]
grad res [ 0. -0. -0. -0. -0. -0.  1.  1.]
imgrad 2 3.877131477680459
eps squared 9.002564686415939
returned et 9.002564686415939
after try except 3.0004274172884
Fidelity 1.0
True error 0.0
actual energy (3.3306690738754696e-16+0j)
trained_en (3.3306690738754696e-16+0j)
nat grad result [-2.99380219e-02 -1.51479245e-02  3.1

nat grad result [ 1.62839981e+00  7.96722778e-02 -1.57364353e-01  1.41313070e-03
  1.19137416e-01  5.91708495e-02  1.42054026e+00  1.42041758e+00]
nat grad result [ 1.87437191  0.14551492 -0.34759046  0.00640829  0.19413228  0.09749462
  1.28670719  1.28831833]
hsquared 11.125821787485021
dtdt 1.9448594406760145
metric [[ 0.25   0.     0.    -0.     0.001  0.    -0.    -0.   ]
 [ 0.     0.25  -0.    -0.    -0.     0.242 -0.    -0.   ]
 [ 0.    -0.     0.017 -0.     0.    -0.     0.    -0.   ]
 [-0.     0.    -0.     0.     0.     0.     0.     0.001]
 [ 0.001 -0.     0.     0.     0.25  -0.064 -0.     0.001]
 [ 0.     0.242 -0.     0.    -0.064  0.25   0.001 -0.   ]
 [-0.    -0.     0.     0.    -0.     0.001  0.25   0.064]
 [-0.    -0.    -0.     0.001  0.001 -0.     0.064  0.25 ]]
nat grad result [ 1.874  0.146 -0.348  0.006  0.194  0.097  1.287  1.288]
grad res [ 1.044e+00  1.000e-03 -7.450e-01  4.000e-03  4.000e-03  4.000e-03
  8.160e-01  8.150e-01]
imgrad 2 4.316976681919566
eps s

nat grad result [ 6.17185555 -0.67881877 -1.55749014  0.00943188 -0.39617697  0.03682486
  0.18029299  0.72910419]
nat grad result [ 6.35391876 -0.78628828 -0.45197103 -0.00919336 -0.30653371 -0.07204986
  0.06063135  0.46869467]
hsquared 11.394861400024526
dtdt 10.351382246583551
metric [[ 0.25  -0.     0.    -0.    -0.003  0.     0.004  0.   ]
 [ 0.     0.25  -0.    -0.     0.001  0.059  0.001  0.001]
 [ 0.    -0.     0.236 -0.    -0.001 -0.    -0.005 -0.008]
 [-0.     0.    -0.     0.    -0.004  0.    -0.003 -0.005]
 [-0.003  0.001 -0.001 -0.004  0.25  -0.164 -0.     0.179]
 [ 0.     0.059 -0.     0.    -0.164  0.25   0.179  0.   ]
 [ 0.004  0.001 -0.005 -0.003 -0.     0.179  0.25   0.164]
 [ 0.     0.001 -0.008 -0.005  0.179  0.     0.164  0.25 ]]
nat grad result [ 6.354 -0.786 -0.452 -0.009 -0.307 -0.072  0.061  0.469]
grad res [ 3.193e+00 -4.020e-01 -2.200e-01 -3.000e-03 -2.300e-02 -7.000e-03
  1.950e-01  1.740e-01]
imgrad 2 20.80730627030126
eps squared 0.9389373763068178
return

nat grad result [ 4.77201708 -0.28492021  1.48657124  0.24066955  0.5795739  -0.21659102
 -0.83930415 -0.91670577]
nat grad result [ 4.56985329e+00  5.63752320e-02  6.45781320e-04  2.24445019e-01
  7.85142459e-01 -6.39845979e-02 -7.01054250e-01 -8.56827703e-01]
hsquared 11.768683395485427
dtdt 5.126134855995131
metric [[ 0.25  -0.     0.    -0.    -0.06   0.     0.022  0.   ]
 [-0.     0.25   0.    -0.    -0.    -0.25  -0.     0.008]
 [ 0.     0.     0.    -0.    -0.    -0.    -0.001  0.   ]
 [-0.    -0.    -0.     0.016  0.    -0.002  0.001 -0.063]
 [-0.06  -0.    -0.     0.     0.25   0.002  0.    -0.001]
 [ 0.    -0.25  -0.    -0.002  0.002  0.25  -0.001 -0.   ]
 [ 0.022 -0.    -0.001  0.001  0.    -0.001  0.248 -0.002]
 [ 0.     0.008  0.    -0.063 -0.001 -0.    -0.002  0.239]]
nat grad result [ 4.57e+00  5.60e-02  1.00e-03  2.24e-01  7.85e-01 -6.40e-02 -7.01e-01
 -8.57e-01]
grad res [ 2.206  0.047 -0.021  0.115 -0.132 -0.056 -0.144 -0.439]
imgrad 2 10.485704935060955
eps squared 6

In [26]:
test = ~StateFn(ansatz)@Hamiltonian@StateFn(ansatz)
print(test.assign_parameters(dict(zip(ansatz.ordered_parameters, init_param_values))).eval())

0j


In [27]:
print(test)

ComposedOp([
  CircuitMeasurement(
       ┌───────────────┐┌───────────────┐     ┌───────────────┐┌───────────────┐
  q_0: ┤ RZ(-1.0*θ[6]) ├┤ RY(-1.0*θ[4]) ├──■──┤ RZ(-1.0*θ[2]) ├┤ RY(-1.0*θ[0]) ├
       ├───────────────┤├───────────────┤┌─┴─┐├───────────────┤├───────────────┤
  q_1: ┤ RZ(-1.0*θ[7]) ├┤ RY(-1.0*θ[5]) ├┤ X ├┤ RZ(-1.0*θ[3]) ├┤ RY(-1.0*θ[1]) ├
       └───────────────┘└───────────────┘└───┘└───────────────┘└───────────────┘
  ),
  3.0 * ZZ
  + 1.0 * XZ
  + 1.0 * ZX,
  CircuitStateFn(
       ┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
  q_0: ┤ RY(θ[0]) ├┤ RZ(θ[2]) ├──■──┤ RY(θ[4]) ├┤ RZ(θ[6]) ├
       ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤
  q_1: ┤ RY(θ[1]) ├┤ RZ(θ[3]) ├┤ X ├┤ RY(θ[5]) ├┤ RZ(θ[7]) ├
       └──────────┘└──────────┘└───┘└──────────┘└──────────┘
  )
])


In [28]:
Hamiltonian

PauliSumOp(SparsePauliOp([[False, False,  True,  True],
               [False,  True,  True, False],
               [ True, False, False,  True]],
              coeffs=[3.+0.j, 1.+0.j, 1.+0.j]), coeff=1.0)